<a href="https://colab.research.google.com/github/wothmag07/genai-bootcamp/blob/main/finetuning_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY


In [ ]:
from openai import OpenAI
client = OpenAI()
tasks = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "List the FAANG companies?"}])

print(tasks.choices[0].message.content.replace('**', ''))

The term FAANG refers to five high-performing technology companies in the stock market. The companies included in FAANG are:

1. Facebook (now known as Meta Platforms, Inc.)
2. Apple Inc.
3. Amazon.com, Inc.
4. Netflix, Inc.
5. Google (now under the parent company Alphabet Inc.)

These companies are known for their significant impact on technology, media, and the economy.


In [ ]:
import json

with open('test.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

data

[{'messages': [{'role': 'system',
    'content': 'You are a customer support agent for a smartphone company whose primary goal is to help users with issues they are experiencing with their smartphones. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to smartphones.'},
   {'role': 'user', 'content': 'What warranty does a smartphone come with?'},
   {'role': 'assistant',
    'content': 'Smartphones typically come with a one-year limited warranty covering manufacturer defects. For specific warranty details, please refer to your product documentation or the official website.'}]},
 {'messages': [{'role': 'system',
    'content': 'You are a customer support agent for a smartphone company whose primary goal is to help users with issues they are experiencing with their smartphones. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to smartphones.'}

In [ ]:
data[0]

{'messages': [{'role': 'system',
   'content': 'You are a customer support agent for a smartphone company whose primary goal is to help users with issues they are experiencing with their smartphones. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to smartphones.'},
  {'role': 'user', 'content': 'What warranty does a smartphone come with?'},
  {'role': 'assistant',
   'content': 'Smartphones typically come with a one-year limited warranty covering manufacturer defects. For specific warranty details, please refer to your product documentation or the official website.'}]}

In [ ]:
clientt = OpenAI()
clientt.files.create(
  file=open("/content/test.jsonl", "rb"),
  purpose='fine-tune'
)
files = clientt.files.list()
files

SyncCursorPage[FileObject](data=[FileObject(id='file-WEZuYcqhSVpEZutwnKfcdA', bytes=6429, created_at=1751050770, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None), FileObject(id='file-X9rzVysp8ZtcPF5B5ivYUy', bytes=6429, created_at=1751049925, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None), FileObject(id='file-NeXdkXSKyq6f1itDPoZjBY', bytes=6429, created_at=1751049897, filename='test.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)], has_more=False, object='list', first_id='file-WEZuYcqhSVpEZutwnKfcdA', last_id='file-NeXdkXSKyq6f1itDPoZjBY')

In [ ]:
training_file_id = files.data[0].id
suffix_name = "mobile-customer-support"

clientt.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name
)

clientt.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-RhaetwYipqFlXB4ZGL6so6T9', created_at=1751051273, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-vZm3pGgq0ZOAcBgsFIi63Agy', result_files=[], seed=325820778, status='validating_files', trained_tokens=None, training_file='file-WEZuYcqhSVpEZutwnKfcdA', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix='mobile-customer-support', usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=False, object='list')

In [ ]:
for job in clientt.fine_tuning.jobs.list():
    print(job.fine_tuned_model)

None


In [ ]:
for job in clientt.fine_tuning.jobs.list():
    print(job.fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal:mobile-customer-support:Bn8oFO4x


In [ ]:
task2 = clientt.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:mobile-customer-support:Bn8oFO4x",
    messages=[{"role": "system", "content": "You are a customer support agent for a smartphone company whose primary goal is to help users with issues they are experiencing with their smartphones. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to smartphones."},
        {
            "role": "user",
            "content": "What warranty does a smartphone come with?"
        }])

print(task2.choices[0].message.content.replace('**', ''))

Smartphones typically come with a one-year limited warranty covering manufacturer defects. For specific warranty details, please refer to your product documentation or the official website.


In [ ]:
task3 = clientt.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:mobile-customer-support:Bn8oFO4x",
    messages=[{"role": "system", "content": "You are a customer support agent for a smartphone company whose primary goal is to help users with issues they are experiencing with their smartphones. You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to smartphones."},
        {
            "role": "user",
            "content": "How much discount will you provide for this new mobile?"
        }])

print(task3.choices[0].message.content.replace('**', ''))

Discounts and promotions vary. Please check the official website or sign up for the newsletter to receive the latest information on discounts.
